In [1]:
import os
from dotenv import find_dotenv, load_dotenv
from fredapi import Fred
import pandas as pd
import pandas_datareader as pdr
import sqlite3
import logging

%matplotlib inline
import matplotlib.pyplot as plt

load_dotenv(find_dotenv())
fred_api_key = os.environ.get('FRED_API_KEY')
fred = Fred(api_key=fred_api_key)

In [2]:
ticker = 'CYS'
series_ids = ['SP500', 'DGS10', 'DGS5', 'USD3MTD156N', 'USD1WKD156N', 'FF']

In [3]:
conn = sqlite3.connect(os.environ.get('DB_PATH'))


In [4]:
for id in series_ids:
    df = fred.get_series_info(id).to_frame().T
    try:
        df.to_sql('data_series', conn, if_exists='append', index=False)
    except sqlite3.IntegrityError as e:
        logging.warning(e)
        continue

In [5]:
for row in conn.execute('select id, title from data_series'):
    print(row)

('SP500', 'S&P 500')
('DGS10', '10-Year Treasury Constant Maturity Rate')
('DGS5', '5-Year Treasury Constant Maturity Rate')
('USD3MTD156N', '3-Month London Interbank Offered Rate (LIBOR), based on U.S. Dollar')
('USD1WKD156N', '1-Week London Interbank Offered Rate (LIBOR), based on U.S. Dollar')
('FF', 'Effective Federal Funds Rate')


In [6]:
for id in series_ids:
    q = conn.execute("SELECT value FROM data where ds_id=?", [str(id)])
    if q.fetchone() is not None:
        print('{0} is already in the DB!'.format(id))
    else:
        print('{0} not found...'.format(id))

SP500 not found...
DGS10 not found...
DGS5 not found...
USD3MTD156N not found...
USD1WKD156N not found...
FF not found...


In [7]:
for id in series_ids:
    q_test = conn.execute("SELECT value FROM data where ds_id=?", [str(id)])
    if q.fetchone() is not None:
        print('{0} is already in the DB!'.format(id))
    else:
        query = conn.execute("SELECT observation_start FROM data_series WHERE id=?", [str(id)])
        data = fred.get_series(id, observation_start=query.fetchone()[0])
        df = data.to_frame(name='value')
        df.rename(columns={'index':'date'}, inplace=True)
        df['ds_id'] = id
        df.dropna(inplace=True)
        print(df.tail())
        df.to_sql('data', conn, if_exists='append', index=True, index_label='date')

              value  ds_id
2017-12-01  2642.22  SP500
2017-12-04  2639.44  SP500
2017-12-05  2629.57  SP500
2017-12-06  2629.27  SP500
2017-12-07  2636.98  SP500
            value  ds_id
2017-11-30   2.42  DGS10
2017-12-01   2.37  DGS10
2017-12-04   2.37  DGS10
2017-12-05   2.36  DGS10
2017-12-06   2.33  DGS10
            value ds_id
2017-11-30   2.14  DGS5
2017-12-01   2.13  DGS5
2017-12-04   2.15  DGS5
2017-12-05   2.15  DGS5
2017-12-06   2.11  DGS5
              value        ds_id
2017-11-24  1.46763  USD3MTD156N
2017-11-27  1.47725  USD3MTD156N
2017-11-28  1.47882  USD3MTD156N
2017-11-29  1.48063  USD3MTD156N
2017-11-30  1.48738  USD3MTD156N
              value        ds_id
2017-11-24  1.20341  USD1WKD156N
2017-11-27  1.20106  USD1WKD156N
2017-11-28  1.20219  USD1WKD156N
2017-11-29  1.20500  USD1WKD156N
2017-11-30  1.20688  USD1WKD156N
            value ds_id
2017-11-08   1.16    FF
2017-11-15   1.16    FF
2017-11-22   1.16    FF
2017-11-29   1.16    FF
2017-12-06   1.15    FF


In [8]:
df = pdr.data.DataReader(ticker, 'yahoo')
# We need to add the CYS series metadata to data_series before we can add the raw data
#conn.execute("""INSERT INTO data_series (
#                frequency_short,
#                id,
#                last_updated,
#                observation_start,
#                observation_end) VALUES 
#                (?, ?, ?, ?, ?)""", 
#                (df.index.freq, str(ticker), df.index[-1].to_pydatetime(), df.index[0].to_pydatetime(), df.index[-1].to_pydatetime()))


df['ds_id'] = ticker
df.rename(columns={'Close': 'value'}, inplace=True)
df.dropna(inplace=True)
print(df.tail())
df.drop(columns=['Open', 'High', 'Low', 'Adj Close', 'Volume'], inplace=True)
df.to_sql('data', conn, if_exists='append', index=True, index_label='date')

            Open  High   Low  value  Adj Close  Volume ds_id
Date                                                        
2017-12-01  8.09  8.16  8.02   8.14       8.14  993200   CYS
2017-12-04  8.19  8.27  8.18   8.26       8.26  795900   CYS
2017-12-05  8.25  8.26  8.19   8.23       8.23  623300   CYS
2017-12-06  8.23  8.27  8.20   8.22       8.22  618800   CYS
2017-12-07  8.21  8.29  8.21   8.29       8.29  778900   CYS


In [9]:
q = conn.execute("SELECT frequency, id FROM data_series")
print(q.fetchall())

[('Daily', 'SP500'), ('Daily', 'DGS10'), ('Daily', 'DGS5'), ('Daily', 'USD3MTD156N'), ('Daily', 'USD1WKD156N'), ('Weekly, Ending Wednesday', 'FF')]
